# 1) Introduction

L’objectif du notebook est de pré traiter les données issues de l’outil d’export de données de Stack Overflow-
"stackexchange explorer", qui recense un grand nombre de données authentiques de la plateforme afin de détecter
les sujets et générer des tags.

## 1.1. Étapes de prétraitement

**1) Suppression du bruit**

1. Suppression du formatage HTML
2. Suppression des contractions
3. La correction orthographique
4. Mettre en minuscule le texte

**2) Suppression des caractères simples**

1. Suppression de la ponctuation, des caractères spéciaux et des nombres
2. Suppression d'un seul caractère (facultatif et spécifique)

**3) Suppression de StopWords**

1. Suppression du mot le plus fréquent
2. Suppression d'un certain type de mot (facultatif et spécifique)

**4) Stemming / Lemmatisation**

1. Stemming
2. Lemmatisation


Ce pré-processus est utilisé pour effectuer une simple détection de sujet (LDA, NMF, etc.) ou une classification,
des informations nécessaires à certaines analyses peuvent être perdues.

 

1.1.1 Vocabulaire
Voici une petite liste de concepts utilisés dans ce cahier.

Tokenize : "Processus de conversion d'une chaîne en une liste de sous-chaînes, appelées tokens."

Normalisation du texte : "Processus de transformation d'un texte en une seule forme canonique qu'il n'aurait
peut-être pas eu auparavant (par exemple, mettre en minuscule le texte, supprimer les contractions, correction
orthographique, stemming / lemmatisation, etc.). La normalisation du texte nécessite de savoir quel type de texte
doit être normalisée et comment elle doit être traitée par la suite ; il n’existe pas de procédure de
normalisation universelle. "

Suppression du bruit : "Processus de suppression de tout élément susceptible interferer avec votre analyse
(par exemple, suppression du code HTML, mettre en minuscule le texte, suppression de la ponctuation / du caractère
spécial, etc.)

Stemming: "Processus de réduction des mots à leur racine , base ou forme de racine - généralement une forme de
mot écrit ("fishing", "fished", and "fisher" to the stem "fish")."

Lemmatisation : "Processus de regroupement des formes fléchies d'un mot afin qu'elles puissent être analysées
comme un seul élément, identifié par le lemme du mot, ou par la forme du dictionnaire (ie : "walking" to "walk",
 "better" to "good")."

StopWord : "Mots qui sont filtrés avant ou après le traitement des données en langage naturel (texte). Les mots
d'arrêt font généralement référence aux mots les plus courants dans une langue (des mots comme "The","a", etc. en
anglais)."

# 2) Libraries and Dataset 

In [1]:
! pip install bs4
! pip install contractions
! pip install autocorrect 

In [2]:
# generic librairies
import pandas as pd

# Text librairies
import re
from bs4 import BeautifulSoup
import nltk
from nltk.tokenize import ToktokTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import contractions
from autocorrect import Speller

In [3]:
# https://numpy.org/devdocs/user/basics.types.html

dtypes_questions = {'Id':'int32', 'Score': 'int16', 'Title': 'string', 'Body': 'string', 'Tags': 'string'}

In [4]:
%%time

nrows=10000

df_questions_python = pd.read_csv('input/QueryResults python score more than 20.csv',
                           usecols=dtypes_questions.keys(),
                           encoding = "ISO-8859-1",
                           dtype=dtypes_questions,
                           nrows=nrows
                          )

df_questions_r = pd.read_csv('input/QueryResults r score more than 20.csv',
                           usecols=['Id', 'Score', 'Title', 'Body', 'Tags'],
                           encoding = "ISO-8859-1",
                           dtype=dtypes_questions,
                           nrows=nrows
                          )

df_questions = pd.concat([df_questions_python,df_questions_r])
df_questions['Title_raw']=df_questions['Title']
df_questions['Body_raw']=df_questions['Body']
df_questions = df_questions.sample(frac=1).reset_index(drop=True)

CPU times: user 200 ms, sys: 14.8 ms, total: 214 ms
Wall time: 214 ms


In [5]:
df_questions[['Title', 'Body', 'Tags']] = df_questions[[
    'Title', 'Body','Tags'
]].applymap(lambda x: str(x).encode("utf-8", errors='surrogatepass').decode(
    "ISO-8859-1", errors='surrogatepass'))

In [6]:
df_questions.dtypes

Id            int32
Title        object
Body         object
Tags         object
Score         int16
Title_raw    string
Body_raw     string
dtype: object

In [7]:
spell = Speller()
token = ToktokTokenizer()
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()
charac = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~0123456789'
stop_words = set(stopwords.words("english"))
adjective_tag_list = {'JJ', 'JJR', 'JJS', 'RBR', 'RBS'}  # List of Adjective's tag from nltk package


**NLTK Tag list**
List of tag use in the tagger (pos_tag function) from NLTK:
https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html

In [8]:
df_questions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13913 entries, 0 to 13912
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Id         13913 non-null  int32 
 1   Title      13913 non-null  object
 2   Body       13913 non-null  object
 3   Tags       13913 non-null  object
 4   Score      13913 non-null  int16 
 5   Title_raw  13913 non-null  string
 6   Body_raw   13913 non-null  string
dtypes: int16(1), int32(1), object(3), string(2)
memory usage: 625.1+ KB


In [9]:
df_questions['qTitle'] = df_questions['Tags'].apply(lambda x : len((x).split(" ")))

# 3) Suppression du bruit

La suppression du bruit consiste à supprimer tout ce qui peut interférer avec votre analyse de texte. C'est comme l'étape de nettoyage des données pour un projet ML classique.

## 3.1. Suppression du code HTML

In [10]:
df_questions['Body'][10]

'<p>I have a string containing an environment variable, e.g. </p>\n\n<pre><code>my_path = \'$HOME/dir/dir2\'\n</code></pre>\n\n<p>I want parse the string, looking up the variable and replacing it in the string:</p>\n\n<pre><code>print "HOME =",os.environ[\'HOME\']\nmy_expanded_path = parse_string(my_path)\nprint "PATH =", my_expanded_path\n</code></pre>\n\n<p>So I should see the output:</p>\n\n<pre><code>HOME = /home/user1\n\nPATH = /home/user1/dir/dir2\n</code></pre>\n\n<p>Is there an elegant way to do that in Python?</p>\n\n<p>thanks!</p>\n\n<p>Conor</p>\n'

In [11]:
%%time

# Analyser la question et le titre puis renvoyer uniquement le texte
df_questions['Body'] = df_questions['Body'].apply(
    lambda x: BeautifulSoup(x, 'html.parser').get_text())
df_questions['Title'] = df_questions['Title'].apply(
    lambda x: BeautifulSoup(x, 'html.parser').get_text())


CPU times: user 6.62 s, sys: 47 ms, total: 6.66 s
Wall time: 6.66 s


BeautifulSoup nous permet de supprimer efficacement la plupart du code html mais pas tout.

In [12]:
df_questions['Body'][10]

'I have a string containing an environment variable, e.g. \nmy_path = \'$HOME/dir/dir2\'\n\nI want parse the string, looking up the variable and replacing it in the string:\nprint "HOME =",os.environ[\'HOME\']\nmy_expanded_path = parse_string(my_path)\nprint "PATH =", my_expanded_path\n\nSo I should see the output:\nHOME = /home/user1\n\nPATH = /home/user1/dir/dir2\n\nIs there an elegant way to do that in Python?\nthanks!\nConor\n'

Nous devons donc supprimer le reste ici.

In [13]:
def clean_text(text):
    """ mettre la déf de la fonction"""
    text = re.sub(r"\'", "'", text) # match all literal apostrophe pattern then replace them by a single whitespace
    text = re.sub(r"\n", " ", text) # match all literal Line Feed (New line) pattern then replace them by a single whitespace
    text = re.sub(r"\xa0", " ", text) # match all literal non-breakable space pattern then replace them by a single whitespace
    text = re.sub('\s+', ' ', text) # match all one or more whitespace then replace them by a single whitespace
    text = text.strip(' ')
    return text


In [14]:
%%time

df_questions['Title'] = df_questions['Title'].apply(lambda x: clean_text(x))
df_questions['Body'] = df_questions['Body'].apply(lambda x: clean_text(x))

CPU times: user 642 ms, sys: 0 ns, total: 642 ms
Wall time: 641 ms


In [15]:
df_questions['Body'][10]

'I have a string containing an environment variable, e.g. my_path = \'$HOME/dir/dir2\' I want parse the string, looking up the variable and replacing it in the string: print "HOME =",os.environ[\'HOME\'] my_expanded_path = parse_string(my_path) print "PATH =", my_expanded_path So I should see the output: HOME = /home/user1 PATH = /home/user1/dir/dir2 Is there an elegant way to do that in Python? thanks! Conor'

Nous devons aussi traiter la colonne des tags


## 3.2. Suppression des contractions

In [16]:
def expand_contractions(text):
    """développer les mots raccourcis, e.g. 'don't' to 'do not'"""
    text = contractions.fix(text)
    return text

In [17]:
%%time

df_questions['Title'] = df_questions['Title'].apply(lambda x: expand_contractions(x))
df_questions['Body'] = df_questions['Body'].apply(lambda x: expand_contractions(x))

CPU times: user 669 ms, sys: 0 ns, total: 669 ms
Wall time: 668 ms


In [18]:
df_questions['Body'][10]

'I have a string containing an environment variable, e.g. my_path = \'$HOME/dir/dir2\' I want parse the string, looking up the variable and replacing it in the string: print "HOME =",os.environ[\'HOME\'] my_expanded_path = parse_string(my_path) print "PATH =", my_expanded_path So I should see the output: HOME = /home/user1 PATH = /home/user1/dir/dir2 Is there an elegant way to do that in Python? thanks! Conor'

## 3.3. La correction orthographique

Pour 1000 entrées cette correction prends 10 minutes

In [19]:
def autocorrect(text):
    words_id = token.tokenize(text)
    words_correct = [spell(i) for i in words_id]
    return ' '.join(map(str, words_correct)) # Return the text untokenize

#df_questions['Title'] = df_questions['Title'].apply(lambda x: autocorrect(x))
#df_questions['Body'] = df_questions['Body'].apply(lambda x: autocorrect(x))

## 3.4. Mettre en minuscule le texte

Je choisis d'abaisser le texte après le paquet de contractions car celui-ci peut remettre des lettres majuscules lors de la suppression des contractions. La mise en minuscule du texte est une étape classique et utile de la suppression du bruit ou de la normalisation du texte car elle réduit le vocabulaire, normalise le texte et ne coûte presque rien.

In [20]:
%%time

df_questions['Title'] = df_questions['Title'].str.lower()
df_questions['Body'] = df_questions['Body'].str.lower()
df_questions['Tags'] = df_questions['Tags'].str.lower()

CPU times: user 22.3 ms, sys: 0 ns, total: 22.3 ms
Wall time: 21.8 ms


In [21]:
df_questions['Body'][10]

'i have a string containing an environment variable, e.g. my_path = \'$home/dir/dir2\' i want parse the string, looking up the variable and replacing it in the string: print "home =",os.environ[\'home\'] my_expanded_path = parse_string(my_path) print "path =", my_expanded_path so i should see the output: home = /home/user1 path = /home/user1/dir/dir2 is there an elegant way to do that in python? thanks! conor'

# 4) Suppression des caractères

## 4.1. Suppression de la ponctuation, des caractères spéciaux et des nombres

TOUS les caractères non alphabétiques ont été supprimés (y compris la ponctuation, les nombres et les caractères spéciaux). Ainsi, je ne considère pas les mots importants qui peuvent contenir des caractères spéciaux (comme "C #" en programmation).

In [22]:
def remove_punctuation_and_number(text):
    """remove all punctuation and number"""
    return text.translate(str.maketrans(" ", " ", charac)) 

def remove_non_alphabetical_character(text):
    """remove all non-alphabetical character"""
    text = re.sub("[^a-z]+", " ", text) # remove all non-alphabetical character
    text = re.sub("\s+", " ", text) # remove whitespaces left after the last operation
    return text

In [23]:
%%time

df_questions['Title'] = df_questions['Title'].apply(lambda x: remove_punctuation_and_number(x))
df_questions['Body'] = df_questions['Body'].apply(lambda x: remove_punctuation_and_number(x))

CPU times: user 147 ms, sys: 13 µs, total: 147 ms
Wall time: 147 ms


In [24]:
%%time

df_questions['Title'] = df_questions['Title'].apply(lambda x: remove_non_alphabetical_character(x))
df_questions['Body'] = df_questions['Body'].apply(lambda x: remove_non_alphabetical_character(x))

CPU times: user 917 ms, sys: 0 ns, total: 917 ms
Wall time: 917 ms


In [25]:
df_questions['Body'][10]

'i have a string containing an environment variable eg mypath homedirdir i want parse the string looking up the variable and replacing it in the string print home osenvironhome myexpandedpath parsestringmypath print path myexpandedpath so i should see the output home homeuser path homeuserdirdir is there an elegant way to do that in python thanks conor'

## 4.2. Suppression de la présence d'un seul caractère


Je choisis de supprimer un seul caractère car lorsque nous faisons de la programmation, nous utilisons souvent un seul caractère alphabétique comme nom de variable ("x", "y", "z", etc.). Et j'ai observé que lorsque j'ai essayé de détecter des sujets sans les supprimer, j'ai trouvé beaucoup de sujets avec eux! Et même un sujet que je pourrais nommer "Nom de variable" ...

In [26]:
def remove_single_letter(text):
    """remove single alphabetical character"""
    text = re.sub(r"\b\w\b", "", text) # remove all single letter
    text = re.sub("\s+", " ", text) # remove whitespaces left after the last operation
    text = text.strip(" ")
    return text

In [27]:
%%time

#df_questions['Title'] = df_questions['Title'].apply(lambda x: remove_single_letter(x))
#df_questions['Body'] = df_questions['Body'].apply(lambda x: remove_single_letter(x))

# nous ne pouvons pas supprimer les single letters car nous voulons garder la lettre R !!

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.48 µs


In [28]:
df_questions['Body'][10]

'i have a string containing an environment variable eg mypath homedirdir i want parse the string looking up the variable and replacing it in the string print home osenvironhome myexpandedpath parsestringmypath print path myexpandedpath so i should see the output home homeuser path homeuserdirdir is there an elegant way to do that in python thanks conor'

# 5) Suppression des stopwords

## 5.1. Removing most frequent words

Supprimer les mots les plus fréquents est une étape classique de la NLP. Les mots les plus fréquents n'ajoutent pas beaucoup d'informations dans la plupart des cas (puisqu'ils sont dans presque toutes les phrases). En les supprimant, vous créez plus d'"espace" pour les autres mots qui peuvent avoir des informations plus utiles.
Vous pouvez utiliser des listes prédéfinies à partir de bibliothèques telles que SciKit-Learn, NLTK et autres. Mais sachez que ces listes peuvent être plus problématiques qu'utiles (en particulier la liste scikit-learn, voir [Stop Word Lists in Free Open-source Software Packages](https://www.aclweb.org/anthology/W18-2502.pdf) pour plus d'informations).

In [29]:
def remove_stopwords(text):
    """remove common words in english by using nltk.corpus's list"""
    words_idx = token.tokenize(text)
    filtered = [i for i in words_idx if not i in stop_words]
    
    return ' '.join(map(str, filtered)) # Return the text untokenize

In [30]:
%%time

df_questions['Title'] = df_questions['Title'].apply(lambda x: remove_stopwords(x))
df_questions['Body'] = df_questions['Body'].apply(lambda x: remove_stopwords(x))

CPU times: user 2.23 s, sys: 3.17 ms, total: 2.23 s
Wall time: 2.23 s


In [31]:
df_questions['Body'][10]

'string containing environment variable eg mypath homedirdir want parse string looking variable replacing string print home osenvironhome myexpandedpath parsestringmypath print path myexpandedpath see output home homeuser path homeuserdirdir elegant way python thanks conor'

## 5.2. Suppression d'adjectives

Je choisis de supprimer les adjectifs en plus de la liste NLTK. Pourquoi ? Tout simplement parce que lorsque j'ai d'abord essayé de faire une détection de sujet dans un cahier suivant celui-ci et cela améliore ma détection de sujet. Je pensais aussi que les adjectifs n'ajouteraient aucune information utile. En même temps, je pourrais aussi supprimer des verbes avec le même raisonnement. Mais je ne l'ai pas fait parce que l'ensemble de données StackOverflow concerne la programmation. Et en programmation, nous avons beaucoup de verbes, ou de mots qui peuvent être interprétés comme un verbe, qui peuvent être importants ("return", "get", "request", "replace", etc.).

In [32]:

# noinspection PyTypeChecker
def remove_by_tag(text, undesired_tag):
    """remove all words by using ntk tag (adjectives, verbs, etc.)"""
    words_idx = token.tokenize(text) # Tokenize each words
    words_tagged = nltk.pos_tag(tokens=words_idx, tagset=None, lang='eng') # Tag each words and return a list of tuples (e.g. ("have", "VB"))
    filtered = [i[0] for i in words_tagged if i[1] not in undesired_tag] # Select all words that don't have the undesired tags
    
    return ' '.join(map(str, filtered)) # Return the text untokenize

In [33]:
%%time
df_questions['Title'] = df_questions['Title'].apply(
    lambda x: remove_by_tag(x, adjective_tag_list))
df_questions['Body'] = df_questions['Body'].apply(
    lambda x: remove_by_tag(x, adjective_tag_list))

CPU times: user 42.9 s, sys: 134 ms, total: 43.1 s
Wall time: 43.1 s


In [34]:
df_questions['Body'][10]

'string containing environment mypath homedirdir want parse string looking replacing string print home osenvironhome myexpandedpath parsestringmypath print path myexpandedpath see output home homeuser path homeuserdirdir way thanks conor'

# 6) Stemming / Lemmatisation

Le Stemming et la Lemmatisation sont des opérations qui :
- peuvent améliorer votre temps de calcul en réduisant votre vocabulaire
- aider à généraliser plus facilement en regroupant les mots (ex: "suis", "sont", "être", etc. seront transformés en "être" pour la lemmatisation)

## 6.1. Stemming

Je n'ai pas choisi d'utiliser le stemming ici, mais l'on doit toujours envisager cette alternative car elle est beaucoup moins coûteuse.

Le stemming est le processus de réduction des mots fléchis à leur racine mot, base ou forme de racine - généralement une forme de mot écrit ("fishing", "fished", and "fisher" à la racine "fish"). Il fonctionne généralement en supprimant l'affixe d'un mot. Un affixe peut être un suffixe ou un préfixe (par exemple «-ed», «-ing», etc.). C'est simple mais ne fonctionnera pas lorsque le mot est "irrégulier" ("ran" et "run"). C'est une opération plus simple que la lemmatisation, qui peut suffire dans certains cas, mais peut faire trop d'erreurs dans d'autres cas.

In [35]:
words = ["program", "programs", "programer", "programing", "programers"]
  
for w in words:
    print(w, " : ", stemmer.stem(w))

program  :  program
programs  :  program
programer  :  program
programing  :  program
programers  :  program


In [36]:
def stem_text(text):
    """Stem the text"""
    words_idx = nltk.word_tokenize(text) # tokenize the text then return a list of tuple (token, nltk_tag)
    stemmed_text = []
    for word in words_idx:
        stemmed_text.append(stemmer.stem(word)) # Stem each words
    return " ".join(stemmed_text) # Return the text untokenize

In [37]:
# %%time

# df_questions['Title'] = df_questions['Title'].apply(lambda x: stem_text(x))
# df_questions['Body'] = df_questions['Body'].apply(lambda x: stem_text(x))

## 6.2. Lemmatisation

Comme dit au début, la lemmatisation est le processus de remplacement d'un mot par son lemma (forme canonique ou forme dictionnaire). Mais dans certains cas, un lemmatiseur peut ne pas être en mesure de trouver la bonne racine si vous ne précisez pas le type de mot comme vous pouvez le voir ci-dessous.

In [38]:
print(lemmatizer.lemmatize("stripes", "v"))
print(lemmatizer.lemmatize("stripes", "n"))  
print(lemmatizer.lemmatize("are"))
print(lemmatizer.lemmatize("are", "v"))

strip
stripe
are
be


Une façon de contourner ce problème consiste à utiliser un marqueur et à passer le type de mot dans la fonction lemmatise. MAIS c'est vraiment coûteux. La mise en tige ou une simple lemmatisation à cet égard est bien plus efficace.

In [39]:
def lemmatize_text(text):
    """Lemmatize the text by using tag """
    
    tokens_tagged = nltk.pos_tag(nltk.word_tokenize(text))  # tokenize the text then return a list of tuple (token, nltk_tag)
    lemmatise_text = []
    for word, tag in tokens_tagged:
        if tag.startswith('J'):
            lemmatise_text.append(lemmatizer.lemmatize(word,'a')) # Lemmatise adjectives. Not doing anything since we remove all adjective
        elif tag.startswith('V'):
            lemmatise_text.append(lemmatizer.lemmatize(word,'v')) # Lemmatise verbs
        elif tag.startswith('N'):
            lemmatise_text.append(lemmatizer.lemmatize(word,'n')) # Lemmatise nouns
        elif tag.startswith('R'):
            lemmatise_text.append(lemmatizer.lemmatize(word,'r')) # Lemmatise adverbs
        else:
            lemmatise_text.append(lemmatizer.lemmatize(word)) # If no tags has been found, perform a non specific lemmatisation
    return " ".join(lemmatise_text) # Return the text untokenize

In [40]:
%%time

df_questions['Title'] = df_questions['Title'].apply(lambda x: lemmatize_text(x))
df_questions['Body'] = df_questions['Body'].apply(lambda x: lemmatize_text(x))

CPU times: user 47.6 s, sys: 87.6 ms, total: 47.7 s
Wall time: 47.7 s


In [41]:
df_questions['Body'][10]

'string contain environment mypath homedirdir want parse string look replace string print home osenvironhome myexpandedpath parsestringmypath print path myexpandedpath see output home homeuser path homeuserdirdir way thanks conor'

# 7) Feature engineering

L'utilisation du titre et du corps en même temps donne de bien meilleurs résultats pour la détection des sujets.

In [42]:
df_questions['Text'] = df_questions['Title'] + ' ' + df_questions['Body']
df_questions['Text_raw'] = df_questions['Title_raw'] + ' ' + df_questions['Body_raw']

# 8) Dataframe avec les Tags

In [43]:
df_questions['Tags'][10]

'<python><posix>'

In [44]:
def clean_split_tag(text):
    """ 
        fonction pour traiter tous les tags en supprimant les caracteres HTML
    """
    text = re.sub(r"<", "", text) # match all literal apostrophe pattern then replace them by a single whitespace
    text = re.sub(r">", ",", text)# match all literal Line Feed (New line) pattern then replace them by a single whitespace
    text = re.sub(r" ", "_", text)
    text = text.strip(',')
    text = text.split(',')
    #text = re.sub(r",", " ", text)
    #text = text.rstrip('>')
    return text

In [45]:
%%time

df_questions['Tags'] = df_questions['Tags'].apply(lambda x: clean_split_tag(x))

CPU times: user 111 ms, sys: 8.04 ms, total: 119 ms
Wall time: 118 ms


In [46]:
tags_list = df_questions[['Id','Tags']].explode('Tags')
print(tags_list.head(10))

def untokenize(words_id):
    return ' '.join(map(str, words_id)) # Return the text untokenize

df_questions['Tags'] = df_questions['Tags'].apply(lambda x: untokenize(x))

         Id               Tags
0   3604587             python
0   3604587  string-formatting
1   1089662                 c#
1   1089662             python
1   1089662        compression
1   1089662               zlib
2   1185634             python
2   1185634          algorithm
3  15259547             python
3  15259547                  r


In [47]:
df_questions['Tags']

0                                 python string-formatting
1                               c# python compression zlib
2                                         python algorithm
3                               python r pandas data.table
4                               python proxy urllib2 socks
                               ...                        
13908                                               python
13909                  python pandas dataframe replace nan
13910                             python django ironpython
13911    python asynchronous twisted addition arithmeti...
13912                                          python list
Name: Tags, Length: 13913, dtype: object

# 9) Exportation des données

In [48]:
print(df_questions.head(10),'\n \n',tags_list.head(10))
#df_questions.drop(['Tags'], axis=1).to_csv('df_questions_fullclean.csv', encoding='utf-8')
df_questions.to_csv('df_questions_fullclean.csv', encoding='utf-8')
tags_list.to_csv('tags_list.csv', encoding='utf-8')

         Id                                     Title  \
0   3604587     use string format show zero precision   
1   1089662     python inflate deflate implementation   
2   1185634            solve mastermind guessing game   
3  15259547                      sum pandas aggregate   
4   2317849                     use sock proxy urllib   
5  14899506  display message json object could decode   
6  26405895     match fuzzy match string two datasets   
7   5084743         print pretty string output python   
8   5594933                    fabric cd manager work   
9  28983292            apply family really vectorized   

                                                Body  \
0  try represent number lead trail width include ...   
1  interfacing server require data send deflate a...   
2  would create solve follow puzzle mastermind ch...   
3  recently make switch r python trouble get use ...   
4                   use sock proxy download web page   
5  python code load data long compli